In [2]:
# Instala o spark 
!pip3 install pyspark

     |████████████████████████████████| 204.2MB 63kB/s 
     |████████████████████████████████| 204kB 46.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=0fab0a169bdbbba83633f48c60ec072d0639ee89dd1f926b3808db9ab4713193
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [3]:
!pyspark

Python 3.6.9 (default, Jul 17 2020, 12:50:27) 
[GCC 8.4.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
20/10/27 22:59:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.0.1
      /_/

Using Python version 3.6.9 (default, Jul 17 2020 12:50:27)
SparkSession available as 'spark'.
>>> quit
Use quit() or Ctrl-D (i.e. EOF) to exit
>>> quit()


# Dataset 1 - MovieLens (Small)

## Importando as bibliotecas da aplicação e configurando o contexto / Execução do Job 

In [4]:
from pyspark import SparkConf, SparkContext
import collections #Do Python

In [5]:
conf = SparkConf().setMaster("local").setAppName("olamundo")
sc = SparkContext(conf= conf)

In [10]:
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

--2020-10-27 23:08:25--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K   965KB/s    in 1.0s    

2020-10-27 23:08:26 (965 KB/s) - ‘ml-latest-small.zip’ saved [978202/978202]



In [11]:
!unzip /content/ml-latest-small.zip

Archive:  /content/ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [18]:
# Carrega o conjunto de dados dentro do Spark
# Estrutura RDD
linhas = sc.textFile("/content/ml-latest-small/ratings.csv")

In [19]:
type(linhas)

pyspark.rdd.RDD

## Explorando

In [20]:
def separar_notas(linha):
  campos = linha.split(",")
  return campos[2]

In [24]:
#Carrega os campos indicivuais e pega os campos de nota
notas = linhas.map(separar_notas)

In [25]:
#Mesma funcionalidade anterior, umas com função anonima
notas2 = linhas.map(lambda linha: linha.split(',')[2]) #Lambda é uma funcao em python que não tem nome

In [26]:
saida1 = notas.countByValue()

In [28]:
saida1.items() #Retorna um dicionario com os valores

dict_items([('rating', 1), ('4.0', 26818), ('5.0', 13211), ('3.0', 20047), ('2.0', 7551), ('1.0', 2811), ('4.5', 8551), ('3.5', 13136), ('2.5', 5550), ('0.5', 1370), ('1.5', 1791)])

In [29]:
saida2 = notas2.countByValue()

In [30]:
saida2.items()

dict_items([('rating', 1), ('4.0', 26818), ('5.0', 13211), ('3.0', 20047), ('2.0', 7551), ('1.0', 2811), ('4.5', 8551), ('3.5', 13136), ('2.5', 5550), ('0.5', 1370), ('1.5', 1791)])

In [33]:
# Colocando os elementos da saída em ordem
from operator import itemgetter

saida_ordenada = collections.OrderedDict(sorted(saida1.items(), 
                                                reverse = True,
                                                key=itemgetter(1))) # odena pela quantidade de ocorrencias

In [34]:
saida_ordenada

OrderedDict([('4.0', 26818),
             ('3.0', 20047),
             ('5.0', 13211),
             ('3.5', 13136),
             ('4.5', 8551),
             ('2.0', 7551),
             ('2.5', 5550),
             ('1.0', 2811),
             ('1.5', 1791),
             ('0.5', 1370),
             ('rating', 1)])

In [35]:
for key,value in saida_ordenada.items():
  print(key,value)

4.0 26818
3.0 20047
5.0 13211
3.5 13136
4.5 8551
2.0 7551
2.5 5550
1.0 2811
1.5 1791
0.5 1370
rating 1


In [36]:
%%file primeiro_prog_spark.py
#Importar as bibliotecas da aplicação
from pyspark import SparkConf, SparkContext
import collections #Do Python

#Configurar a forma de execução do Job e o contexto
conf = SparkConf().setMaster("local").setAppName("olaMundo")
sc = SparkContext(conf=conf)

#Carrega o conjunto de dados dentro do Spark
#Estrutura RDD
linhas = sc.textFile("/content/ml-latest-small/ratings.csv")

#Mesma funcionalidade anterior, mas com funcao anonima
notas = linhas.map(lambda linha : linha.split(',')[2])

saida = notas.countByValue()

#Coloca os elementos da saída em ordem
from operator import itemgetter
saida_ordenada = collections.OrderedDict(sorted(saida.items(), 
                                                reverse=True,
                                                key=itemgetter(1)))

for key,value in saida_ordenada.items():
  print(key, value)
  #print(key, ';', value) #traz a saída com ;

Writing primeiro_prog_spark.py


In [38]:
#Rodando no console da máquinha
!spark-submit primeiro_prog_spark.py > saida.txt

20/10/27 23:27:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
20/10/27 23:27:48 INFO SparkContext: Running Spark version 3.0.1
20/10/27 23:27:48 INFO ResourceUtils: ==============================================================
20/10/27 23:27:48 INFO ResourceUtils: Resources for spark.driver:

20/10/27 23:27:48 INFO ResourceUtils: ==============================================================
20/10/27 23:27:48 INFO SparkContext: Submitted application: olaMundo
20/10/27 23:27:48 INFO SecurityManager: Changing view acls to: root
20/10/27 23:27:48 INFO SecurityManager: Changing modify acls to: root
20/10/27 23:27:48 INFO SecurityManager: Changing view acls groups to: 
20/10/27 23:27:48 INFO SecurityManager: Changing modify acls groups to: 
20/10/27 23:27:48 INFO SecurityManager: SecurityManager: authentication disabled; ui acls di

# Dataset 2 - movieLens Completo

In [39]:
!wget http://files.grouplens.org/datasets/movielens/ml-latest.zip

--2020-10-27 23:53:25--  http://files.grouplens.org/datasets/movielens/ml-latest.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277113433 (264M) [application/zip]
Saving to: ‘ml-latest.zip’

ml-latest.zip       100%[===================>] 264.28M  16.8MB/s    in 17s     

2020-10-27 23:53:42 (15.6 MB/s) - ‘ml-latest.zip’ saved [277113433/277113433]

unzip:  cannot find or open /content/mo-latest.zip, /content/mo-latest.zip.zip or /content/mo-latest.zip.ZIP.


In [40]:
!unzip /content/ml-latest.zip

Archive:  /content/ml-latest.zip
   creating: ml-latest/
  inflating: ml-latest/links.csv     
  inflating: ml-latest/tags.csv      
  inflating: ml-latest/genome-tags.csv  
  inflating: ml-latest/ratings.csv   
  inflating: ml-latest/README.txt    
  inflating: ml-latest/genome-scores.csv  
  inflating: ml-latest/movies.csv    


In [41]:
%%file segundo_prog_spark.py
#Importar as bibliotecas da aplicação
from pyspark import SparkConf, SparkContext
import collections #Do Python

#Configurar a forma de execução do Job e o contexto
conf = SparkConf().setMaster("local").setAppName("olaMundo")
sc = SparkContext(conf=conf)

#Carrega o conjunto de dados dentro do Spark
#Estrutura RDD
linhas = sc.textFile("/content/ml-latest/ratings.csv")

#Mesma funcionalidade anterior, mas com funcao anonima
notas = linhas.map(lambda linha : linha.split(',')[2])

saida = notas.countByValue()

#Coloca os elementos da saída em ordem
from operator import itemgetter
saida_ordenada = collections.OrderedDict(sorted(saida.items(), 
                                                reverse=True,
                                                key=itemgetter(1)))

for key,value in saida_ordenada.items():
  print(key,';', value)

Writing segundo_prog_spark.py


In [ ]:
#Roda no console da máquina
!spark-submit segundo_prog_spark.py > saida_2.csv

In [43]:
#Manipulação dos dados com DataSets
from pyspark.sql import SparkSession 

In [44]:
spark = SparkSession.builder.appName('segunda-versao').getOrCreate()

In [45]:
#Abrindo o conjunto de dados
dados = spark.read.csv('/content/ml-latest-small/ratings.csv', header = True, inferSchema=True)

In [46]:
#Visualizando o DF
dados.head(5)

[Row(userId=1, movieId=1, rating=4.0, timestamp=964982703),
 Row(userId=1, movieId=3, rating=4.0, timestamp=964981247),
 Row(userId=1, movieId=6, rating=4.0, timestamp=964982224),
 Row(userId=1, movieId=47, rating=5.0, timestamp=964983815),
 Row(userId=1, movieId=50, rating=5.0, timestamp=964982931)]

In [47]:
#Mostrando todo o DF (Por padrão até 20)
dados.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



In [48]:
#Exibe o schema
dados.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [49]:
#Forçando outro tipo de Schema (configurando)
from pyspark.sql.types import (StructField, StringType, IntegerType, DoubleType, StructType)


In [53]:
#Construindo o Schema de dados
data_schema = [StructField('userId', StringType()), 
               StructField('movieId', IntegerType()),
               StructField('rating', DoubleType()),
               StructField('timestamp',IntegerType())]

In [54]:
#Criando o Schema
schema = StructType(fields=data_schema) 

In [56]:
#Lendo o Conjunto de dados após a mudança do Schema
dados = spark.read.csv('/content/ml-latest-small/ratings.csv', header = True, schema=schema)

In [57]:
dados.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [58]:
dados.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



In [59]:
dados['userId']

Column<b'userId'>

In [60]:
#Retornando um DF
dados.select('userId')

DataFrame[userId: string]

In [62]:
dados.select('rating').describe().show()

+-------+------------------+
|summary|            rating|
+-------+------------------+
|  count|            100836|
|   mean| 3.501556983616962|
| stddev|1.0425292390606342|
|    min|               0.5|
|    max|               5.0|
+-------+------------------+



In [63]:
dados.select(['rating','timestamp']).describe().show()

+-------+------------------+--------------------+
|summary|            rating|           timestamp|
+-------+------------------+--------------------+
|  count|            100836|              100836|
|   mean| 3.501556983616962|1.2059460873684695E9|
| stddev|1.0425292390606342|2.1626103599513078E8|
|    min|               0.5|           828124615|
|    max|               5.0|          1537799250|
+-------+------------------+--------------------+



In [65]:
resultado = dados.filter(dados['rating'] >= 4)

In [67]:
resultado.describe().show()

+-------+------------------+------------------+-------------------+--------------------+
|summary|            userId|           movieId|             rating|           timestamp|
+-------+------------------+------------------+-------------------+--------------------+
|  count|             48580|             48580|              48580|               48580|
|   mean| 316.4663235899547|18952.660354055166|  4.359952655413751|1.1989993179792097E9|
| stddev|181.62850777837113| 35451.49892492841|0.43172426790053686|2.1787508781316867E8|
|    min|                 1|                 1|                4.0|           828124615|
|    max|                99|            193609|                5.0|          1537799250|
+-------+------------------+------------------+-------------------+--------------------+



In [68]:
#Lendo o Conjunto de dados grande
dados = spark.read.csv('/content/ml-latest/ratings.csv', header = True, schema=schema)

In [69]:
dados.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [71]:
dados.describe().show()

+-------+------------------+-----------------+------------------+--------------------+
|summary|            userId|          movieId|            rating|           timestamp|
+-------+------------------+-----------------+------------------+--------------------+
|  count|          27753444|         27753444|          27753444|            27753444|
|   mean|141942.01557064414|18487.99983414671|3.5304452124932677|1.1931218549319255E9|
| stddev| 81707.40009148984| 35102.6252474677|1.0663527502319696|2.1604822852233613E8|
|    min|                 1|                1|               0.5|           789652004|
|    max|             99999|           193886|               5.0|          1537945149|
+-------+------------------+-----------------+------------------+--------------------+



# Dataset 3 - Benefícios Concedidos (Previdência)

In [72]:
!wget --no-check-certificate http://dadosabertos.dataprev.gov.br/storage/f/2020-08-20T17%3A51%3A05.439Z/beneficios-concedidos-12-2018.csv

--2020-10-28 00:27:57--  http://dadosabertos.dataprev.gov.br/storage/f/2020-08-20T17%3A51%3A05.439Z/beneficios-concedidos-12-2018.csv
Resolving dadosabertos.dataprev.gov.br (dadosabertos.dataprev.gov.br)... 200.152.32.122
Connecting to dadosabertos.dataprev.gov.br (dadosabertos.dataprev.gov.br)|200.152.32.122|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://dadosabertos.dataprev.gov.br/storage/f/2020-08-20T17%3A51%3A05.439Z/beneficios-concedidos-12-2018.csv [following]
--2020-10-28 00:27:58--  https://dadosabertos.dataprev.gov.br/storage/f/2020-08-20T17%3A51%3A05.439Z/beneficios-concedidos-12-2018.csv
Connecting to dadosabertos.dataprev.gov.br (dadosabertos.dataprev.gov.br)|200.152.32.122|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 60455337 (58M) [text/csv]
Saving to: ‘beneficios-concedidos-12-2018.csv’

beneficios-concedid 100%[===================>]  57.65M

##Média de valores de benefícios por estado

In [82]:
data = spark.read.csv('/content/beneficios-concedidos-12-2018.csv', header=True, inferSchema=True, encoding= 'ISO-8859-1', sep=';')

In [81]:
data.printSchema()

root
 |-- Competência concessão: string (nullable = true)
 |-- Espécie: string (nullable = true)
 |-- CID2: string (nullable = true)
 |-- CID3: string (nullable = true)
 |-- Despacho: string (nullable = true)
 |-- Dt Nascimento: string (nullable = true)
 |-- Sexo.: string (nullable = true)
 |-- Clientela: string (nullable = true)
 |-- Mun Resid: string (nullable = true)
 |-- Vínculo dependentes: string (nullable = true)
 |-- Forma Filiação: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- Qt SM RMI: string (nullable = true)



In [84]:
data.show(2)

+---------------------+--------------------+----+-------+--------------------+-------------+--------+---------+--------------------+-------------------+-----------------+--------+---------+
|Competência concessão|             Espécie|CID2|   CID3|            Despacho|Dt Nascimento|   Sexo.|Clientela|           Mun Resid|Vínculo dependentes|   Forma Filiação|      UF|Qt SM RMI|
+---------------------+--------------------+----+-------+--------------------+-------------+--------+---------+--------------------+-------------------+-----------------+--------+---------+
|           2018-12-01|Pensão por Morte ...|   0|Zerados|Concessao Decorre...|   22/06/2005|Feminino|   Urbano|     02043-AL-Maceió|              Filho|        Empregado| Alagoas|    4,326|
|           2018-12-01|Auxílio Salario M...|   0|Zerados|    Concessao Normal|   14/08/1994|Feminino|    Rural|03038-AM-São Paul...|              Filho|Segurado Especial|Amazonas|        1|
+---------------------+--------------------+----+-

In [85]:
df = data.select(['UF', 'Qt SM RMI'])

In [86]:
df.show()

+--------+---------+
|      UF|Qt SM RMI|
+--------+---------+
| Alagoas|    4,326|
|Amazonas|        1|
|   Bahia|    1,586|
|   Bahia|        1|
|   Bahia|        1|
|   Bahia|        1|
|   Bahia|    1,499|
|   Bahia|        1|
|   Bahia|        1|
|   Bahia|        1|
|   Bahia|    1,112|
|   Bahia|        1|
|   Bahia|        1|
|   Bahia|        1|
|   Bahia|    1,917|
|   Ceará|    1,325|
|   Ceará|    1,079|
|   Ceará|    2,849|
|   Ceará|        1|
|   Ceará|        1|
+--------+---------+
only showing top 20 rows



In [90]:
#Trocando a virgula por ponto
from pyspark.sql.functions import translate
from pyspark.sql.types import DoubleType

df_translate = df.withColumn('Qt SM RMI', translate('Qt SM RMI', ',', '.'))
df = df_translate.withColumn('Qt SM RMI', df_translate['Qt SM RMI'].cast(DoubleType()))

In [91]:
df.printSchema()

root
 |-- UF: string (nullable = true)
 |-- Qt SM RMI: double (nullable = true)



In [93]:
df.describe().show()

+-------+---------+-----------------+
|summary|       UF|        Qt SM RMI|
+-------+---------+-----------------+
|  count|   339434|           339434|
|   mean|     null|1.477993262313144|
| stddev|     null|0.994561212552488|
|    min|     Acre|            0.005|
|    max|Tocantins|          262.599|
+-------+---------+-----------------+



In [94]:
#Agrupando as informações por coluna
df.groupBy('UF').mean().show()

+-------------------+------------------+
|                 UF|    avg(Qt SM RMI)|
+-------------------+------------------+
|     Santa Catarina|1.5099096048360068|
| Mato Grosso do Sul|1.4401450272964633|
|              Goiás|1.4242774689532858|
|        Mato Grosso| 1.457399332591769|
|              Ceará|1.2149186999528967|
|     Espírito Santo|1.4552418495297814|
|              Piauí|1.1552044579533944|
|             Paraná|1.4631905558288363|
|            Alagoas| 1.232052440290758|
|              Bahia|1.2516893867264103|
|            Roraima|1.3369999999999986|
|   Distrito Federal| 1.399875889572658|
|         Pernambuco|1.3109834572490695|
|           Amazonas|1.4209408425455632|
|  Rio Grande do Sul|1.4859394904458625|
|               Acre|1.5148905325443776|
|Rio Grande do Norte|1.2911911942098917|
|            Sergipe|1.2859992239037632|
|          São Paulo| 1.774125415754047|
|     Rio de Janeiro|1.6405266633941804|
+-------------------+------------------+
only showing top

In [95]:
df.groupBy('UF').max().show()

+-------------------+--------------+
|                 UF|max(Qt SM RMI)|
+-------------------+--------------+
|     Santa Catarina|          8.05|
| Mato Grosso do Sul|         5.776|
|              Goiás|        14.357|
|        Mato Grosso|         6.008|
|              Ceará|         8.326|
|     Espírito Santo|         8.065|
|              Piauí|         6.871|
|             Paraná|         8.996|
|            Alagoas|         8.475|
|              Bahia|         8.326|
|            Roraima|         5.849|
|   Distrito Federal|         6.242|
|         Pernambuco|          7.34|
|           Amazonas|        12.153|
|  Rio Grande do Sul|        26.962|
|               Acre|       262.599|
|Rio Grande do Norte|         8.858|
|            Sergipe|          9.79|
|          São Paulo|        11.352|
|     Rio de Janeiro|         15.31|
+-------------------+--------------+
only showing top 20 rows



In [108]:
from pyspark.sql.functions import max,min
max_beneficio = df.select(max(df['Qt Sm RMI'])).collect()[0][0] #devolve um conjunto de linhas a qual eu quer a primeira linha com o primeiro valor
min_beneficio = df.select(min(df['Qt Sm RMI'])).collect()[0][0]

In [105]:
#Econtrando o dono do maior beneficio
data.filter(data['Qt SM RMI'] ==  str(max_beneficio).replace('.',',')).show()

+---------------------+--------------------+----+-------+--------------------+-------------+---------+---------+--------------------+-------------------+--------------+----+---------+
|Competência concessão|             Espécie|CID2|   CID3|            Despacho|Dt Nascimento|    Sexo.|Clientela|           Mun Resid|Vínculo dependentes|Forma Filiação|  UF|Qt SM RMI|
+---------------------+--------------------+----+-------+--------------------+-------------+---------+---------+--------------------+-------------------+--------------+----+---------+
|           2018-12-01|Amparo Social ao ...|   0|Zerados|Concessao Decorre...|   26/01/1952|Masculino|   Urbano|24004-AC-Cruzeiro...|      Não Informado|  Desempregado|Acre|  262,599|
+---------------------+--------------------+----+-------+--------------------+-------------+---------+---------+--------------------+-------------------+--------------+----+---------+



In [109]:
#Econtrando o dono do menor beneficio 
data.filter(data['Qt SM RMI'] ==  str(min_beneficio).replace('.',',')).show()

+---------------------+--------------------+----+-------+--------------------+-------------+---------+---------+-------------------+-------------------+--------------+--------------+---------+
|Competência concessão|             Espécie|CID2|   CID3|            Despacho|Dt Nascimento|    Sexo.|Clientela|          Mun Resid|Vínculo dependentes|Forma Filiação|            UF|Qt SM RMI|
+---------------------+--------------------+----+-------+--------------------+-------------+---------+---------+-------------------+-------------------+--------------+--------------+---------+
|           2018-12-01|Aposentadoria Inv...|   0|Zerados|Concessao Decorre...|   17/02/1963|Masculino|   Urbano|07049-ES-São Mateus|      Não Informado|  Desempregado|Espírito Santo|    0,005|
+---------------------+--------------------+----+-------+--------------------+-------------+---------+---------+-------------------+-------------------+--------------+--------------+---------+

